## Regression Analysis for Estimated Salary

In [1]:
# Importing required libraries

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
import pickle

In [2]:
data = pd.read_csv("dataset/ChurnPrediction.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### **Data Preprocessing**

In [3]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1)

In [4]:
# Encoding categorical variables

label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender']) # Performs fit and transform in one go
data.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1


In [5]:
# One-Hot Encoding on Geography column

one_hot_encoder = OneHotEncoder()
geography_encoder = one_hot_encoder.fit_transform(data[['Geography']]).toarray()
print(geography_encoder)

[[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [6]:
# Converting to dataframe

geography_encoded_df = pd.DataFrame(geography_encoder, columns = one_hot_encoder.get_feature_names_out(['Geography']))
geography_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [7]:
# Dropping the old Geography column
data = data.drop(["Geography"], axis = 1)

# Combining One Hot Encoded columns with the original data
data = pd.concat([data, geography_encoded_df], axis = 1)
data.head(4)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0


### **Train-Test Split**

In [8]:
# Independent and Dependent features

X = data.drop('EstimatedSalary', axis = 1)
Y = data['EstimatedSalary']

In [9]:
# Training and Testing Split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [10]:
# Scaling the features

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) # 
X_test = scaler.transform(X_test) # tranform for preventing data leakage and for ensuring your model generalizes well

### **ANN Regression**

In [11]:
# Importing required libraries

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
import pickle

In [12]:
# Building our ANN model

model = Sequential([
    Dense(64, activation = 'relu', input_shape = (X_train.shape[1],)), # First Hidden Layer connected with inputs 
    Dense(32, activation = 'relu'),  # Second Hidden Layer
    Dense(1) # Output Layer for Regression
])

/opt/anaconda3/envs/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Initialising optimizer and loss for compilation

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)

loss = tf.keras.losses.MeanAbsoluteError()

In [14]:
# Compiling the model

model.compile(optimizer = optimizer, loss = loss, metrics = ['mae'])

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Setting up the TensorBoard for catching logs

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [17]:
# Setting up Early Stopping for avoiding overfitting
# Early stopping will monitor the validation loss and stop training if it doesn't improve for 10 epochs

early_stopping_callback = EarlyStopping(monitor = "val_loss", patience = 10, restore_best_weights = True)

In [18]:
history = model.fit(
    X_train, Y_train, validation_data = (X_test, Y_test), epochs = 100,

    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 911us/step - loss: 82998.0156 - mae: 82998.0156 - val_loss: 52201.8711 - val_mae: 52201.8711
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 596us/step - loss: 50534.8398 - mae: 50534.8398 - val_loss: 50491.6367 - val_mae: 50491.6367
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 599us/step - loss: 49961.3828 - mae: 49961.3828 - val_loss: 50256.5977 - val_mae: 50256.5977
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 616us/step - loss: 49786.2305 - mae: 49786.2305 - val_loss: 50212.4141 - val_mae: 50212.4141
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 611us/step - loss: 49669.7656 - mae: 49669.7656 - val_loss: 50256.5977 - val_mae: 50256.5977
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 635us/step - loss: 49593.1484 - mae: 49593.1484 - val_loss: 50300.9492 - val_mae: 50300.9492
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 618us/step - loss: 49550.0391 - mae: 49550.0391 - val_loss: 50283.2305 - val_mae: 50283.2305
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 

In [19]:
%load_ext tensorboard

In [21]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6008 (pid 10584), started 0:00:12 ago. (Use '!kill 10584' to kill it.)

In [22]:
## Evaluate model on the test data

test_loss, test_mae = model.evaluate(X_test, Y_test)

print(f'Test MAE : {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 454us/step - loss: 50212.4141 - mae: 50212.4141
Test MAE : 50212.4140625


In [23]:
# Saving the model
model.save('models/regression_model.h5')